[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/_HOWTOs/dimensional_analysis.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/_HOWTOs/dimensional_analysis.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/_HOWTOs/dimensional_analysis.ipynb)

### PySDM dimensional analysis HOWTO
- PySDM depends on the [Pint](https://pint.readthedocs.io/en/stable/) package which offers dimensional analysis (physical units checks) of Python code 
- this improves code readibility with expressions such as `p = 1000 * si.hPa`
- using [Pint](https://pint.readthedocs.io/en/stable/), `si` is an instance of [`pint.UnitRegistry`](https://pint.readthedocs.io/en/stable/api/base.html#pint.UnitRegistry) 
- however, for performance reasons, by default PySDM uses a custom drop-in-replacement [`FakeUnitRegistry`](https://open-atmos.github.io/PySDM/PySDM/physics/impl/fake_unit_registry.html#FakeUnitRegistry)
- this way, we keep the readibility advantage, while not incurring any performance overhead
- moreover, this makes the code potentially Numba JIT-compilable!
- we also provide a way to leverage the dimensional analysis benefit for testing purposes
- to this end, the test code can use the [`DimensionalAnalysis`](https://open-atmos.github.io/PySDM/PySDM/physics/dimensional_analysis.html#DimensionalAnalysis) context manager
- code below demonstrate how a single unit-equipped function can be used with and without unit checks 

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

#### sample physics-related code

In [2]:
from PySDM import physics

def code():
    si = physics.si

    p = 1000 * si.hPa
    T = 300 * si.K
    R = 286 * si.J / si.K / si.kg

    rho = p / R / T
    return rho

#### sample unit-unaware usage (default)

In [3]:
result_unit_unaware = code()
print(f"{result_unit_unaware:.2g}")

1.2


#### sample unit-aware usage (e.g., for testing)

In [4]:
from PySDM.physics.dimensional_analysis import DimensionalAnalysis

with DimensionalAnalysis():
    result_unit_aware = code()

assert result_unit_aware.check("[mass] / [volume]")
print(f"{result_unit_aware:.2g}")
print(f"{result_unit_aware.to_base_units():.2g}")

0.012 hectopascal * kilogram / joule
1.2 kilogram / meter ** 3
